In [1]:
import os
import os.path as osp
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.nn.functional as F

import core

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
os.environ["TORCH_HOME"] = "."

In [3]:
model = torch.hub.load('hub/deeplabv3', source="local", model='deeplabv3_resnet50', pretrained=True)

In [4]:
print(model)
print(model.aux_classifier)

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

output['out'] contains the semantic masks, and output['aux'] contains the auxiliary loss values per-pixel. In inference mode, output['aux'] is not useful. So, output['out'] is of shape (N, 21, H, W)

In [5]:
model.aux_classifier[-1] = nn.Conv2d(256, 9, kernel_size=(1, 1), stride=(1, 1))
model.classifier[-1] = nn.Conv2d(256, 9, kernel_size=(1, 1), stride=(1, 1))
stretch = nn.Upsample(scale_factor=(4, 1), mode='bilinear', align_corners=False)
squeeze = nn.Conv2d(5, 3, kernel_size=3, stride=1, padding=1)

In [6]:
import torch
import torch.nn as nn

# 创建一个 64x512 的随机特征图
x = torch.randn(4, 5, 64, 512)  # (batch_size, channels, height, width)

x = stretch(x)
x = squeeze(x)

model.train()
x = model(x)
out, aux = x["out"], x["aux"]

out = F.upsample(out, size=(64, 512))
aux = F.upsample(aux, size=(64, 512))

y = torch.randint(0, 9, (4, 64, 512))
criterion = nn.CrossEntropyLoss()
loss1 = criterion(out, y)
loss2 = criterion(aux, y)

/home/fredom/programfiles/miniconda3/envs/torch/lib/python3.9/site-packages/torch/nn/functional.py:3722: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataloader = torch.utils.data.DataLoader(
    dataset=core.dataset.KITTISpherical(
        "../data", "train",
        core.readconfyaml.read("../conf/data.yaml")
    ),
    batch_size=16,
    shuffle=True,
    num_workers=2
)

model = nn.DataParallel(
    core.model.deeplabv3.DeepLabV3(
        in_channels=5,
        out_channels=9
    )
).to(device)
# model = core.model.deeplabv3.DeepLabV3(
#     in_channels=5,
#     out_channels=9
# ).to(device)

optimizer = torch.optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
criterion = torch.nn.CrossEntropyLoss(
    weight=torch.tensor([0.125, 1.2, 1.1, 0.5, 1.5, 1, 1, 1.2, 1.2], dtype=torch.float32),
    ignore_index=-1
).to(device)

In [8]:
for epoch in range(50):
    for batch_idx, (fmap, gdth) in enumerate(train_dataloader):
        fmap = fmap.to(device).float()
        gdth = gdth.to(device).long()

        pred_out, pred_aux = model(fmap)

        loss_out = criterion(pred_out, gdth)
        loss_aux = criterion(pred_aux, gdth)
        loss = loss_out + 0.5 * loss_aux
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(loss.item())
    scheduler.step()


3.2685232162475586
2.8947298526763916
2.743563413619995
2.3090569972991943
2.821913242340088
2.398554563522339
2.3876466751098633
2.196821689605713
2.074831962585449
2.1899986267089844
1.7227543592453003
1.5937005281448364
1.434555172920227
1.324143886566162
1.09194815158844
1.266557216644287
1.0720535516738892
1.1448367834091187
1.0799620151519775
0.9918667078018188
1.4350110292434692
0.8438413143157959
0.9266127347946167
0.787192165851593
0.8882670998573303
0.8300322890281677
0.9777035713195801
0.8619391918182373
1.159769892692566
0.9598375558853149
0.7376770973205566
0.9119455814361572


KeyboardInterrupt: 